In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

dataset_path = 'AMZNclass_data.csv'
df = pd.read_csv('AMZNclass_data.csv')

# Extract dataset name from the file path
dataset_name = dataset_path.split('/')[-1].split('.')[0] 


In [17]:
df

,Date,Open,High,Low,Close,Volume,MACD,RSI,MA,EMA,SMA,STD,Upper,Lower,PatternClass
0,2000-01-03,4.075000,4.478125,3.952344,4.468750,322352000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
1,2000-01-06,3.565625,3.634375,3.200000,3.278125,375040000,NaN,0.000000,3.382812,3.436285,3.382812,0.104687,3.592187,3.173438,4
2,2000-01-07,3.350000,3.525000,3.309375,3.478125,210108000,NaN,36.363644,3.378125,3.464178,3.378125,0.100000,3.578125,3.178125,6
3,2000-01-10,3.628125,3.631250,3.278125,3.459375,295158000,NaN,34.042539,3.468750,3.460976,3.468750,0.009375,3.487500,3.450000,4
4,2000-01-12,3.393750,3.400000,3.150000,3.178125,216090000,NaN,8.510636,3.257812,3.244970,3.257812,0.079688,3.417188,3.098437,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3370,2022-12-22,85.519997,85.680000,82.250000,83.790001,81431300,-3.342859,20.097895,85.279999,84.640855,85.279999,1.489998,88.259995,82.300003,4
3371,2022-12-23,83.250000,85.779999,82.930000,85.250000,57433700,-3.312041,52.056891,84.520000,85.046952,84.520000,0.730000,85.980000,83.060001,6
3372,2022-12-27,84.970001,85.349998,83.000000,83.040001,57284000,-3.426447,23.545680,84.145000,83.708984,84.145000,1.105000,86.355000,81.935001,4
3373,2022-12-29,82.870003,84.550003,82.550003,84.180000,54995900,-3.461244,65.284476,83.000000,83.603221,83.000000,1.180000,85.360001,80.639999,6


In [19]:


# Load your OHLC dataset with indicators
# Replace 'your_dataset.csv' with the actual file path or DataFrame variable


# Assuming 'PatternClass' column is already defined in your dataset

# Remove rows where PatternClass is 0 (No Pattern)
df = df[df['PatternClass'] != 0]

# Convert 'Date' to datetime and extract relevant features
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# Select features (X) and target variable (y)
X = df.drop(['PatternClass', 'Date', 'Open', 'High', 'Low', 'Close'], axis=1)
selected_features = ['PatternClass', 'Open', 'High', 'Low', 'Close','Volume']  # Replace with your actual feature names
X = df[selected_features]
y = df['PatternClass']
print(X.columns)

# Handle missing values by filling with mean
X = X.fillna(X.mean())

# Check for and handle infinite values
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X = X.fillna(0)  # You can customize the filling strategy based on your data

# Encode any categorical variables if necessary
label_encoder = LabelEncoder()
X_encoded = X.apply(lambda col: label_encoder.fit_transform(col.astype(str)) if col.dtype == 'O' else col)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=2)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the classifiers
rf_classifier = RandomForestClassifier(random_state=100, n_estimators=100)
svm_classifier = SVC(random_state=100)
logreg_classifier = LogisticRegression(random_state=100)
dt_classifier = DecisionTreeClassifier(random_state=100)
knn_classifier = KNeighborsClassifier()
nb_classifier = GaussianNB()
extra_trees_classifier = ExtraTreesClassifier(random_state=100, n_estimators=100)  # Added Extra Trees Classifier

# List of classifiers
classifiers = [rf_classifier, svm_classifier, logreg_classifier, dt_classifier, knn_classifier, nb_classifier, extra_trees_classifier]
classifier_names = ['Random Forest', 'SVM', 'Logistic Regression', 'Decision Tree', 'KNN', 'Naive Bayes', 'Extra Trees']

# Initialize DataFrame to store results
results_df = pd.DataFrame(columns=['Classifier', 'Accuracy', 'F1 Score'])

# Loop through each classifier
for classifier, classifier_name in zip(classifiers, classifier_names):
    # Train the classifier
    classifier.fit(X_train_scaled, y_train)

    # Make predictions on the test set
    y_pred = classifier.predict(X_test_scaled)

     # Evaluate the classifier
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')

    # Append results to the DataFrame
    results_df = results_df.append({'Classifier': classifier_name, 'Accuracy': accuracy, 'F1 Score': f1,'Precision': precision, 'Recall': recall}, ignore_index=True)

# Display the results and name of dataset
print(f"Dataset Name: {dataset_name}")
print()
print(results_df)



Index(['PatternClass', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')


C:\Users\ASUS\AppData\Local\Temp\ipykernel_3700\1011279392.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Classifier': classifier_name, 'Accuracy': accuracy, 'F1 Score': f1,'Precision': precision, 'Recall': recall}, ignore_index=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_3700\1011279392.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Classifier': classifier_name, 'Accuracy': accuracy, 'F1 Score': f1,'Precision': precision, 'Recall': recall}, ignore_index=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_3700\1011279392.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Classifier': classifier_name, 'Accuracy'

Dataset Name: AMZNclass_data

            Classifier  Accuracy  F1 Score  Precision    Recall
0        Random Forest  0.997038  0.996831   0.997056  0.997038
1                  SVM  1.000000  1.000000   1.000000  1.000000
2  Logistic Regression  1.000000  1.000000   1.000000  1.000000
3        Decision Tree  1.000000  1.000000   1.000000  1.000000
4                  KNN  0.998026  0.997937   0.998030  0.998026
5          Naive Bayes  1.000000  1.000000   1.000000  1.000000
6          Extra Trees  1.000000  1.000000   1.000000  1.000000


C:\Users\ASUS\AppData\Local\Temp\ipykernel_3700\1011279392.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Classifier': classifier_name, 'Accuracy': accuracy, 'F1 Score': f1,'Precision': precision, 'Recall': recall}, ignore_index=True)


In [10]:

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Load your OHLC dataset with indicators
# Replace 'your_dataset.csv' with the actual file path or DataFrame variable


# Assuming 'PatternClass' column is already defined in your dataset

# Remove rows where PatternClass is 0 (No Pattern)
df = df[df['PatternClass'] != 0]

# Convert 'Date' to datetime and extract relevant features
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# Select features (X) and target variable (y)
X = df.drop(['PatternClass', 'Date'], axis=1)  # Adjust columns accordingly
y = df['PatternClass']
print(X.columns)
# Handle missing values by filling with mean
X = X.fillna(X.mean())

# Check for and handle infinite values
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X = X.fillna(0)  # You can customize the filling strategy based on your data

# Encode any categorical variables if necessary
label_encoder = LabelEncoder()
X_encoded = X.apply(lambda col: label_encoder.fit_transform(col.astype(str)) if col.dtype == 'O' else col)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=2)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Evaluate the Random Forest model
def evaluate_model(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    conf_matrix = confusion_matrix(y_true, y_pred)
    
    print(f"{model_name} Metrics:")
    print("Accuracy:", accuracy)
    print("F1 Score:", f1)
    print("Confusion Matrix:\n", conf_matrix)
    print()



# Initialize the Support Vector Machine (SVM) Classifier
svm_classifier = SVC(random_state=100)

# Train the SVM classifier
svm_classifier.fit(X_train_scaled, y_train)

# Make predictions on the test set using SVM
y_pred_svm = svm_classifier.predict(X_test_scaled)

# Evaluate the SVM model
evaluate_model(y_test, y_pred_svm, "SVM")


Index(['Open', 'High', 'Low', 'Close', 'Volume', 'MACD', 'RSI', 'MA', 'EMA',
       'SMA', 'STD', 'Upper', 'Lower', 'Year', 'Month', 'Day'],
      dtype='object')
SVM Metrics:
Accuracy: 0.824284304047384
F1 Score: 0.8143931634554755
Confusion Matrix:
 [[  0   6   0   6]
 [  0 409   0  91]
 [  0   3   0   9]
 [  0  63   0 426]]



In [5]:
from sklearn.tree import DecisionTreeClassifier
# Initialize the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=100)

# Train the Decision Tree classifier
dt_classifier.fit(X_train_scaled, y_train)

# Make predictions on the test set using Decision Tree
y_pred_dt = dt_classifier.predict(X_test_scaled)

## Evaluate the Decision Tree model
accuracy_dt = accuracy_score(y_test, y_pred_dt)
f1_dt = f1_score(y_test, y_pred_dt, average='weighted')
conf_matrix_dt = confusion_matrix(y_test, y_pred_dt)

print("\nDecision Tree Metrics:")
print("Accuracy:", accuracy_dt)
print("F1 Score:", f1_dt)
print("Confusion Matrix:\n", conf_matrix_dt)


Decision Tree Metrics:
Accuracy: 0.7591312931885489
F1 Score: 0.7604589877760766
Confusion Matrix:
 [[  1   0   6   0   5]
 [  0   0   0   0   0]
 [  0   1 383  16 100]
 [  0   0   5   0   7]
 [  2   0  96   6 385]]


In [6]:
print(X.columns)

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'MACD', 'RSI', 'MA', 'EMA',
       'SMA', 'STD', 'Upper', 'Lower', 'Year', 'Month', 'Day'],
      dtype='object')


In [77]:
from sklearn.linear_model import LogisticRegression
# Initialize the Logistic Regression Classifier
logreg_classifier = LogisticRegression(random_state=42)

# Train the Logistic Regression classifier
logreg_classifier.fit(X_train_scaled, y_train)

# Make predictions on the test set using Logistic Regression
y_pred_logreg = logreg_classifier.predict(X_test_scaled)

# Evaluate the Logistic Regression model
accuracy_logreg = accuracy_score(y_test, y_pred_logreg)
f1_logreg = f1_score(y_test, y_pred_logreg, average='weighted')
conf_matrix_logreg = confusion_matrix(y_test, y_pred_logreg)

print("\nLogistic Regression Metrics:")
print("Accuracy:", accuracy_logreg)
print("F1 Score:", f1_logreg)
print("Confusion Matrix:\n", conf_matrix_logreg)


Logistic Regression Metrics:
Accuracy: 0.8193484698914116
F1 Score: 0.8095933283147387
Confusion Matrix:
 [[  0   6   0   6]
 [  0 414   0  86]
 [  0   1   0  11]
 [  0  73   0 416]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [78]:
print(X.columns)

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'MACD', 'RSI', 'MA', 'EMA',
       'SMA', 'STD', 'Upper', 'Lower', 'Year', 'Month', 'Day'],
      dtype='object')


In [79]:
from sklearn.neighbors import KNeighborsClassifier

# Initialize the K-Nearest Neighbors Classifier
knn_classifier = KNeighborsClassifier()

# Train the KNN classifier
knn_classifier.fit(X_train_scaled, y_train)

# Make predictions on the test set using KNN
y_pred_knn = knn_classifier.predict(X_test_scaled)

# Evaluate the KNN model
accuracy_knn = accuracy_score(y_test, y_pred_knn)
f1_knn = f1_score(y_test, y_pred_knn, average='weighted')
conf_matrix_knn = confusion_matrix(y_test, y_pred_knn)

print("\nK-Nearest Neighbors Metrics:")
print("Accuracy:", accuracy_knn)
print("F1 Score:", f1_knn)
print("Confusion Matrix:\n", conf_matrix_knn)



K-Nearest Neighbors Metrics:
Accuracy: 0.790720631786772
F1 Score: 0.7812747400499447
Confusion Matrix:
 [[  0   7   0   5]
 [  0 401   0  99]
 [  0   2   0  10]
 [  0  89   0 400]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [80]:
from sklearn.naive_bayes import GaussianNB

# Initialize the Naive Bayes Classifier
nb_classifier = GaussianNB()

# Train the Naive Bayes classifier
nb_classifier.fit(X_train_scaled, y_train)

# Make predictions on the test set using Naive Bayes
y_pred_nb = nb_classifier.predict(X_test_scaled)

# Evaluate the Naive Bayes model
accuracy_nb = accuracy_score(y_test, y_pred_nb)
f1_nb = f1_score(y_test, y_pred_nb, average='weighted')
conf_matrix_nb = confusion_matrix(y_test, y_pred_nb)

print("\nNaive Bayes Metrics:")
print("Accuracy:", accuracy_nb)
print("F1 Score:", f1_nb)
print("Confusion Matrix:\n", conf_matrix_nb)



Naive Bayes Metrics:
Accuracy: 0.6209279368213229
F1 Score: 0.6779381019153647
Confusion Matrix:
 [[  8   0   0   4]
 [135 288   0  77]
 [  4   1   0   7]
 [ 67  89   0 333]]
